# A Data Preparation Experiment (An Altered Titanic DataSet)
In this experiment we see importance of data preparation part in the CRISP-DM standard.
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/CRISP-DM_Process_Diagram.png" width="500"/>
</div>
Image Source: www.wikimedia.org

In [0]:
from pyspark.sql.functions import rand, randn
# it creates a DataFrame with one int column and 9 rows.
dataframe1 = sqlContext.range(0, 9)
dataframe1.show()

+---+
 id|
+---+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
+---+

In [0]:
import pandas as pd
# Here we generate a dataframe with 3 columns, one is just "id", the second has uniform distribution and the last has normal distribution.
dataframe2 =sqlContext.range(0, 9).select("id",rand(seed=110).alias("uniform"), randn(seed=220).alias("normal"))
# as a review you can convert your spark sql dataframe to pandas dataframe in order to use functions from that library
datapandas=dataframe2.toPandas()
dataframe2.show()

+---+--------------------+-------------------+
 id| uniform| normal|
+---+--------------------+-------------------+
 0| 0.6555464624430072|-0.9270945164845644|
 1|0.049125825652604704|-0.1405173341048025|
 2| 0.8132791490429919|-1.7191632213297061|
 3| 0.4918765016830071| 0.7680076309262491|
 4| 0.8725599870191662| 1.347025359206477|
 5| 0.892420081305659|-0.7990582332893195|
 6| 0.0561107864441549|-0.5414593534324789|
 7| 0.7678731430866403| 0.9425321381147357|
 8| 0.12765923921818467|-0.5623417285392092|
+---+--------------------+-------------------+

In [0]:
dataframe2.describe('uniform', 'normal').show()

+-------+--------------------+--------------------+
summary| uniform| normal|
+-------+--------------------+--------------------+
 count| 9| 9|
 mean| 0.5251612417661574|-0.18134102877029099|
 stddev| 0.3571099284034672| 1.0049496759142997|
 min|0.049125825652604704| -1.7191632213297061|
 max| 0.892420081305659| 1.347025359206477|
+-------+--------------------+--------------------+

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()
data = spark.read.csv('/FileStore/tables/titanic_alt.csv',inferSchema=True,header=True)
data.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: double (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- city: string (nullable = true)
-- bmi: integer (nullable = true)

## Data Dictionary
- survival:	Survival	(0 = No, 1 = Yes)
- pclass:	Ticket class	(1 = 1st, 2 = 2nd, 3 = 3rd)
- sex:	Sex	
- Age:	Age in years	
- sibsp:	# of siblings / spouses aboard the Titanic	
- parch:	# of parents / children aboard the Titanic	
- ticket:	Ticket number	
- fare:	Passenger fare	
- cabin:	Cabin number	
- city:	original city of passengers (B = Beijing, D = Delhi, C = Cherbourg, Q = Queenstown, S = Southampton, L = London)
- bmi:	bmi of passenger

Also, in case of unknown value, 888 is assigned to the observations

### The relevant columns (variables) are:
survival, Pclass, sex, Age, sibsp, parch, fare, city

In [0]:
data.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: double (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- city: string (nullable = true)
-- bmi: integer (nullable = true)

In [0]:
# this is not necessary but just as a practice
from pyspark.sql.types import StructField,StringType,IntegerType,DoubleType,StructType
# if the second element is True means it could have null cells
df_schema = StructType([
    StructField('PassengerId', StringType(), True),
    StructField('Survived', IntegerType(), True),
    StructField('Pclass', StringType(), True),
    StructField('Sex', StringType(), True),
    StructField('Age', DoubleType(), True),
    StructField('SibSp', IntegerType(), True),
    StructField('Parch', IntegerType(), True),
    StructField('Ticket', StringType(), True),
    StructField('Fare', DoubleType(), True),
    StructField('Cabin', StringType(), True),
    StructField('city', StringType(), True),
    StructField('bmi', IntegerType(), True)
    ])
data = spark.createDataFrame(data.collect(),schema = df_schema)
data.printSchema()

root
-- PassengerId: string (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- city: string (nullable = true)
-- bmi: integer (nullable = true)

In [0]:
# look at numerical values in a string column
data.show()

+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+----+----+
PassengerId|Survived|Pclass| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|city| bmi|
+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+----+----+
 1| 0| 3.0| male|22.0| 1| 0| A/5 21171| 7.25| null| S| 18|
 2| 1| 1.0|female|38.0| 1| 0| PC 17599|71.2833| C85| C|null|
 3| 1| 3.0|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|null|
 4| 1| 1.0|female|35.0| 1| 0| 113803| 53.1| C123| S| 22|
 5| 0| 3.0| male|35.0| 0| 0| 373450| 8.05| null| S|null|
 6| 0| 3.0| male|null| 0| 0| 330877| 8.4583| null| Q| 21|
 7| 0| 1.0| male|54.0| 0| 0| 17463|51.8625| E46| S|null|
 8| 0| 3.0| male| 2.0| 3| 1| 349909| 21.075| null| S|null|
 9| 1| 3.0|female|27.0| 0| 2| 347742|11.1333| null|null|null|
 10| 1| 2.0|female|14.0| 1| 0| 237736|30.0708| null|null|null|
 11| 1| 3.0|female| 4.0| 1| 1| PP 9549| 16.7| G6|null|null|
 12| 1| 1.0|female|58.0| 0| 0| 113783| 26.55| C103|null|null|
 13| 0| 3.0| male|20.0| 0| 0| A/5. 2151| 8.05| null|null|null|
 14| 0| 3.0| male|39.0| 1| 5| 347082| 31.275| null|null|null|
 15| 0| 3.0|female|14.0| 0| 0| 350406| 7.8542| null| S|null|
 16| 1| 888.0|female|55.0| 0| 0| 248706| 16.0| null| S|null|
 17| 0| 888.0| male| 2.0| 4| 1| 382652| 29.125| null| Q|null|
 18| 1| 888.0| male|null| 0| 0| 244373| 13.0| null| S|null|
 19| 0| 3.0|female|31.0| 1| 0| 345763| 18.0| null| S|null|
 20| 1| 3.0|female|null| 0| 0| 2649| 7.225| null| C|null|
+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+----+----+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import when, count, col
data = data.replace(888, None)
data = data.replace("888.0", None) # it how a number is shown in a string column
df = data.select(['Survived',  'Pclass',  'Sex',  'Age',  'SibSp',  'Parch', 'Fare', 'city', 'bmi'])
na_report=df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])# dimensions of the dataframe
print("Number of Rows: ",df.count() ,"   Number of Columns: ", len(df.columns))
# we should handle the missing values by imputation unless too many of them are emtpy
# na.drop() method is not recommended for avoiding biased except for the dependent variable
na_report.show()

Number of Rows: 891 Number of Columns: 9
+--------+------+---+---+-----+-----+----+----+---+
Survived|Pclass|Sex|Age|SibSp|Parch|Fare|city|bmi|
+--------+------+---+---+-----+-----+----+----+---+
 9| 18| 15|180| 0| 0| 0| 8|852|
+--------+------+---+---+-----+-----+----+----+---+

In [0]:
# Checking emptiness proportion in each column
for field in na_report.schema.fields:
        name = str(field.name)
        na_report = na_report.withColumn(name, round(col(name)/df.count(),4))
na_report.show()

+--------+------+------+-----+-----+-----+----+-----+------+
Survived|Pclass| Sex| Age|SibSp|Parch|Fare| city| bmi|
+--------+------+------+-----+-----+-----+----+-----+------+
 0.0101|0.0202|0.0168|0.202| 0.0| 0.0| 0.0|0.009|0.9562|
+--------+------+------+-----+-----+-----+----+-----+------+

In [0]:
# bmi column has too much empty values, we could drop it
df=df.drop('bmi')
df.show(10)

+--------+------+------+----+-----+-----+-------+----+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|city|
+--------+------+------+----+-----+-----+-------+----+
 0| 3.0| male|22.0| 1| 0| 7.25| S|
 1| 1.0|female|38.0| 1| 0|71.2833| C|
 1| 3.0|female|26.0| 0| 0| 7.925| S|
 1| 1.0|female|35.0| 1| 0| 53.1| S|
 0| 3.0| male|35.0| 0| 0| 8.05| S|
 0| 3.0| male|null| 0| 0| 8.4583| Q|
 0| 1.0| male|54.0| 0| 0|51.8625| S|
 0| 3.0| male| 2.0| 3| 1| 21.075| S|
 1| 3.0|female|27.0| 0| 2|11.1333|null|
 1| 2.0|female|14.0| 1| 0|30.0708|null|
+--------+------+------+----+-----+-----+-------+----+
only showing top 10 rows

In [0]:
from sklearn.impute import SimpleImputer
import pandas as pd

# for statistical imputation we convert the pyspark dataframe into pandas to use functions of this library
# deep=True means it create a new independent copy of data, if it's False, it makes a reference to the data. The default value is False
misswork = df.toPandas().copy(deep=True)
median_imputer = SimpleImputer(strategy='median')
misswork.iloc[:,3:4] = median_imputer.fit_transform(misswork.iloc[:,3:4])

In [0]:
# for categorical variable we consider mode for imputation
# mode().iloc[:1,] we pick the first row of mode because some of categorical variables might have more than one category as the most frequent category
# .iloc[0] is just for convering the produced dataframe
misswork.iloc[:,[1,2,7]]=misswork.iloc[:,[1,2,7]].fillna(misswork.iloc[:,[1,2,7]].mode().iloc[:1,].iloc[0])
# instead of below code, you can convert the dataframe into spark and use na.drop() method
misswork=misswork.dropna(subset=['Survived'])
df=spark.createDataFrame(misswork)
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+------+---+---+-----+-----+----+----+
Survived|Pclass|Sex|Age|SibSp|Parch|Fare|city|
+--------+------+---+---+-----+-----+----+----+
 0| 0| 0| 0| 0| 0| 0| 0|
+--------+------+---+---+-----+-----+----+----+

In [0]:
# The descriptive Statistics of the numerical coolumns
# check if you are suspected in some values
df.select('Age','SibSp','Parch','Fare').describe().show()

+-------+------------------+------------------+-------------------+-----------------+
summary| Age| SibSp| Parch| Fare|
+-------+------------------+------------------+-------------------+-----------------+
 count| 882| 882| 882| 882|
 mean|29.614705215419498|0.5226757369614512|0.43197278911564624|33.57039784580499|
 stddev|14.559726292368623|1.1061927041977806| 1.0613519997452092|52.45018659935543|
 min| -7.0| 0| 0| 0.0|
 max| 120.0| 8| 10| 512.3292|
+-------+------------------+------------------+-------------------+-----------------+

#what issues you can see in above?

In [0]:
df = df.filter((df.Age > 0) )

what is your idea about negative age, number of parent childern (they might too busy to survive, it's subjective), and fare?

In [0]:
quantiles = {
    c: dict(
        zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ["Age", "SibSp","Parch","Fare"]
}
quantiles

Out[32]: {'Age': {'q1': 22.0, 'q3': 35.0},
 'SibSp': {'q1': 0.0, 'q3': 1.0},
 'Parch': {'q1': 0.0, 'q3': 0.0},
 'Fare': {'q1': 7.925, 'q3': 31.275}}

In [0]:
for i in quantiles:
    iqr = quantiles[i]['q3'] - quantiles[i]['q1']
    quantiles[i]['lower_bound'] = quantiles[i]['q1'] - (iqr * 1.5)
    quantiles[i]['upper_bound'] = quantiles[i]['q3'] + (iqr * 1.5)
print(quantiles)

{'Age': {'q1': 22.0, 'q3': 35.0, 'lower_bound': 2.5, 'upper_bound': 54.5}, 'SibSp': {'q1': 0.0, 'q3': 1.0, 'lower_bound': -1.5, 'upper_bound': 2.5}, 'Parch': {'q1': 0.0, 'q3': 0.0, 'lower_bound': 0.0, 'upper_bound': 0.0}, 'Fare': {'q1': 7.925, 'q3': 31.275, 'lower_bound': -27.099999999999998, 'upper_bound': 66.3}}

In [0]:
import pyspark.sql.functions as f
df_clean=df.select(
    "*",
    *[
        f.when(
            f.col(c).between(quantiles[c]['lower_bound'], quantiles[c]['upper_bound']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ["Age", "SibSp","Parch","Fare"]
    ]
)
df_clean.show(10)

+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|city|Age_out|SibSp_out|Parch_out|Fare_out|
+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+
 0.0| 3.0| male|22.0| 1| 0| 7.25| S| 0| 0| 0| 0|
 1.0| 1.0|female|38.0| 1| 0|71.2833| C| 0| 0| 0| 1|
 1.0| 3.0|female|26.0| 0| 0| 7.925| S| 0| 0| 0| 0|
 1.0| 1.0|female|35.0| 1| 0| 53.1| S| 0| 0| 0| 0|
 0.0| 3.0| male|35.0| 0| 0| 8.05| S| 0| 0| 0| 0|
 0.0| 3.0| male|28.0| 0| 0| 8.4583| Q| 0| 0| 0| 0|
 0.0| 1.0| male|54.0| 0| 0|51.8625| S| 0| 0| 0| 0|
 0.0| 3.0| male| 2.0| 3| 1| 21.075| S| 1| 1| 1| 0|
 1.0| 3.0|female|27.0| 0| 2|11.1333| S| 0| 0| 1| 0|
 1.0| 2.0|female|14.0| 1| 0|30.0708| S| 0| 0| 0| 0|
+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+
only showing top 10 rows

In [0]:
from pyspark.sql.functions import col
df_clean=df_clean.withColumn("outliers", col("Age_out")+col("SibSp_out")+col("Parch_out")+col("Fare_out"))
df_clean.show()


+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|city|Age_out|SibSp_out|Parch_out|Fare_out|outliers|
+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+--------+
 0.0| 3.0| male|22.0| 1| 0| 7.25| S| 0| 0| 0| 0| 0|
 1.0| 1.0|female|38.0| 1| 0|71.2833| C| 0| 0| 0| 1| 1|
 1.0| 3.0|female|26.0| 0| 0| 7.925| S| 0| 0| 0| 0| 0|
 1.0| 1.0|female|35.0| 1| 0| 53.1| S| 0| 0| 0| 0| 0|
 0.0| 3.0| male|35.0| 0| 0| 8.05| S| 0| 0| 0| 0| 0|
 0.0| 3.0| male|28.0| 0| 0| 8.4583| Q| 0| 0| 0| 0| 0|
 0.0| 1.0| male|54.0| 0| 0|51.8625| S| 0| 0| 0| 0| 0|
 0.0| 3.0| male| 2.0| 3| 1| 21.075| S| 1| 1| 1| 0| 3|
 1.0| 3.0|female|27.0| 0| 2|11.1333| S| 0| 0| 1| 0| 1|
 1.0| 2.0|female|14.0| 1| 0|30.0708| S| 0| 0| 0| 0| 0|
 1.0| 3.0|female| 4.0| 1| 1| 16.7| S| 0| 0| 1| 0| 1|
 1.0| 1.0|female|58.0| 0| 0| 26.55| S| 1| 0| 0| 0| 1|
 0.0| 3.0| male|20.0| 0| 0| 8.05| S| 0| 0| 0| 0| 0|
 0.0| 3.0| male|39.0| 1| 5| 31.275| S| 0| 0| 1| 0| 1|
 0.0| 3.0|female|14.0| 0| 0| 7.8542| S| 0| 0| 0| 0| 0|
 1.0| 3.0|female|55.0| 0| 0| 16.0| S| 1| 0| 0| 0| 1|
 0.0| 3.0| male| 2.0| 4| 1| 29.125| Q| 1| 1| 1| 0| 3|
 1.0| 3.0| male|28.0| 0| 0| 13.0| S| 0| 0| 0| 0| 0|
 0.0| 3.0|female|31.0| 1| 0| 18.0| S| 0| 0| 0| 0| 0|
 1.0| 3.0|female|28.0| 0| 0| 7.225| C| 0| 0| 0| 0| 0|
+--------+------+------+----+-----+-----+-------+----+-------+---------+---------+--------+--------+
only showing top 20 rows

In [0]:
# dropping outliers
df_clean = df_clean.filter((df_clean.outliers == 0) )
df_clean=df_clean.select(["Survived","Pclass", "Sex", "Age","SibSp","Parch", "Fare","city"])
df.select('Age','SibSp','Parch','Fare').describe().show()

+-------+------------------+------------------+------------------+------------------+
summary| Age| SibSp| Parch| Fare|
+-------+------------------+------------------+------------------+------------------+
 count| 876| 876| 876| 876|
 mean| 29.84151826484018|0.5251141552511416| 0.432648401826484|33.684126826484025|
 stddev|14.347175828840532|1.1091553999505208|1.0642374899873552| 52.60452884314071|
 min| 0.42| 0| 0| 0.0|
 max| 120.0| 8| 10| 512.3292|
+-------+------------------+------------------+------------------+------------------+

In [0]:
import numpy as np
print("proportion of the lost Rows: ",np.round((df.count()-df_clean.count())/df.count(),4))

proportion of the lost Rows: 0.3642

we lost a big proportion, you might get back and change some steps!!

In [0]:
df_clean.registerTempTable("dataclean")
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


In [0]:
# distribution of cities in database
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


In [0]:
# distribution of cities in database for each category of survival
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


In [0]:
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


# look at the previous 3 blocks, what issues we might face in the project? What are your solutions?

In [0]:
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


# look at the previous block, what issues we might face in the project? What is your solution?

In [0]:
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S


#look at the previous block, what issues we might face in the project? What is your solution?

In [0]:
display(sqlContext.sql("select * from dataclean"))

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,city
0.0,3.0,male,22.0,1,0,7.25,S
1.0,3.0,female,26.0,0,0,7.925,S
1.0,1.0,female,35.0,1,0,53.1,S
0.0,3.0,male,35.0,0,0,8.05,S
0.0,3.0,male,28.0,0,0,8.4583,Q
0.0,1.0,male,54.0,0,0,51.8625,S
1.0,2.0,female,14.0,1,0,30.0708,S
0.0,3.0,male,20.0,0,0,8.05,S
0.0,3.0,female,14.0,0,0,7.8542,S
1.0,3.0,male,28.0,0,0,13.0,S
